# Steganography

Wow!  We've provided some of the functions from the last assignment (with some slight modifications) below!

`getRGB()` may take a little while to run for large image files (we cannot resize them without messing up any embedded messages!)

`show_image()` has also been modified to support less-complex greyscale images! (for the extra credit)

Run the cells!

In [1]:
# libraries!
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import copy
import time

In [2]:
def getRGB(filename):
    """ reads a png or jpg file like 'pitzer_grounds.jpg' (a string)
        returns the pixels as a list-of-lists-of-lists
        this is accessible, but not fast: Use small images!
    """
    original = Image.open(filename)
    print(f"Reading image from '{filename}':")
    print(f"  Format: {original.format}\n  Original Size: {original.size}\n  Mode: {original.mode}")
    WIDTH, HEIGHT = original.size
    px = original.load()
    PIXEL_LIST = []
    for r in range(HEIGHT):
        row = []
        for c in range(WIDTH):
            row.append( px[c,r][:3] )
        PIXEL_LIST.append( row )
    return PIXEL_LIST

In [3]:
def set_size(width, height, ax=None):
    """Sets the size of an image when printing in the notebook
       w, h: width, height in inches """
    w = width; h = height
    if not ax: 
        ax=plt.gca()  # gets current axes
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)

In [4]:
# wrapper for matplotlib's imshow function
def show_image( rgbdata, hgt_in=5.42, wid_in=5.42 ):
    """ shows an image whose pixels are in rgbdata 
        note:  rgbdata is a list-of-rows-of-pixels-of-rgb values, _not_ a filename!
            use getRGB() to get this data!
        hgt_in is the desired height (in inches)
        wid_in is the desired width (in inches)
            use set_size() with these parameters
        _but_ the library will not change the aspect ratio (takes the smaller)
        by default, the hgt_in and wid_in are 5.42 in.
        (and feel free to change these!)
    """
    fig, ax = plt.subplots()               # obtains the figure and axes objects
    
    if type(rgbdata[0][0]) == list:
        im = ax.imshow(rgbdata)            # this is matplotlib's call to show an image 
    if type(rgbdata[0][0] == int):
        im = ax.imshow(rgbdata, cmap="gray")
    
    set_size(width=wid_in, height=hgt_in)  # matplotlib will maintain the image's aspect ratio
    ax.axis('off')                         # turns off the axes (in units of pixels)
    plt.show()                             # show the image

In [5]:
def saveRGB( PX, filename ):
    """ saves a list-of-lists-of-lists of rgb pixels (PX) where
        len(PX) == the # of rows
        len(PX[0]) == the # of columns
        len(PX[0][0]) should be 3 (rgb)
    """
    boxed_pixels = PX
    print( 'Starting to save', filename, '...' )
    H = len(PX)
    W = len(PX[0])
    im = Image.new("RGB", (W, H), "black")
    px = im.load()
    for r in range(H):
        for c in range(W):
            bp = boxed_pixels[r][c]
            t = tuple(bp)
            px[c,r] = t
    im.save( filename )
    time.sleep(0.42)   # give the filesystem some time...
    print( filename, "saved." )    

## Test it out

Let's view the first few pixels of the 'flag.png' image

We'll march in "English-reading" order (across the 0th row first, then to the next row)

Remember that each pixel has an r, g, and b value

We will convert each of these values to an 8-bit binary number (ex 42 --> '00101010')

In [6]:
filename = 'flag.png'   # feel free to change this

image_rgb = getRGB(filename)

for row in range(0,1):
    for col in range(0,3):
        print(f"pixel at {row},{col} is {image_rgb[row][col]}")
        for value in image_rgb[row][col]:
            binary = bin(value)[2:]
            nbits = len(binary)    # to make sure we have 8 bits...
            binary = '0'*(8-nbits) + binary
            print(f"   value: {value} is the bits: {binary}")

Reading image from 'flag.png':
  Format: PNG
  Original Size: (625, 415)
  Mode: RGB
pixel at 0,0 is (47, 130, 160)
   value: 47 is the bits: 00101111
   value: 130 is the bits: 10000010
   value: 160 is the bits: 10100000
pixel at 0,1 is (47, 132, 163)
   value: 47 is the bits: 00101111
   value: 132 is the bits: 10000100
   value: 163 is the bits: 10100011
pixel at 0,2 is (45, 134, 166)
   value: 45 is the bits: 00101101
   value: 134 is the bits: 10000110
   value: 166 is the bits: 10100110


# Your task... 

Write some functions to embed and extract messages in an image!

- `desteganographize( image )` 
    + This function will take an image's RGB data as input. 
    + It will then read the last bit of each RGB value (in that order!) 
    + And generate a message of 8-bit strings
    + Until it encounters a '00000000' which signals the end of the message
    
Here's an example: if your 8-bit message is the bits 00101010  (followed by the end-of-message marker 00000000), then you would be able to find those bits in this order:
- The red channel of image[0][0]       would end in the bit 0  (the initial bit)
- The green channel of image[0][0]   would end in the bit 0  (the next bit)
- The blue channel of image[0][0]      would end in the bit 1  (the next bit)
- The red channel of image [0][1]       would end in the bit 0  (the next bit)
- The green channel of image [0][1]   would end in the bit 1  (the next bit)
- The blue channel of image [0][1]      would end in the bit 0  (the next bit)
- The red channel of image [0][2]        would end in the bit 1  (the next bit)
- The green channel of image [0][2]    would end in the bit 0  (the next bit)
- Then, the blue channel of image[0][2]  would end in 0   (the first of 8 0's)
- … and then it would continue in this way for seven more 0 bits, since the message needs to end in 8 zero bits.

Of course, each 8-bit string would only represent _**one**_ character! 

Remember that you can use python's built in `bin()` (`bin(42) = '00101010'`) and `int()` (`int("00101010",2) = 42`) functions to convert back and forth between binary and base-10.

Also recall that Python's `chr()` function can convert between a base-10 number and its corresponding ASCII/unicode character (`chr(42) = '*'`). The function `ord()` will do the opposite (`ord('*') = 42`)


- `steganographize( image, message )`
    + This function will do the opposite of `desteganographize()`!
    + It takes in an existing image's RGB data and a message and changes the image's pixels to embed the message!
    + _Consider_ using the `copy` library to ensure you don't modify the existing image
    + The function then returns the modified image's RGB data

Be sure to include the results of your work!

You can use the `saveRGB()` and `show_image()` functions provided to help you

In [22]:
def desteganographize(image_rgb):
    """Extracts a message hidden in an image's RGB data"""
    
    num_rows = len(image_rgb) 
    num_cols = len(image_rgb[0])
    print(f"There are {num_rows} rows and {num_cols} columns\n")
    sub = ""
    message = ""
    bitStr = ""
    
    for row in range(num_rows):
        # print(f"row is {row}")
        for col in range(num_cols):
            for bit in image_rgb[row][col]: # go through each rgb value 
                # check the least significant number (using even or odd instead of converting to bin)
                if bit % 2 == 0: 
                    bitStr += "0"
                else:
                    bitStr += "1"
    
    # save the previous index of the 8 bits to use as the next starting point            
    prev = 0
    for char in range(8, len(bitStr)-8, 8):
        if bitStr[prev:char] == "00000000": # if the message ends 
            break
        
        # print(bitStr[prev:char])
        chr_num = (int(bitStr[prev:char], 2)) # convert to ASCII letters 
        message += chr(chr_num) # add to the message 
        prev = char
    
    return message

desteganographize(image_rgb)

There are 415 rows and 625 columns

01000011
01100001
01101110
00100000
01100011
01101111
01100110
01100110
01100101
01100101
00100000
01101000
01100101
01101100
01110000
00100000
01111001
01101111
01110101
00100000
01101100
01101001
01110110
01100101
00100000
01101100
01101111
01101110
01100111
01100101
01110010
00111111
00001010
00001010
01000011
01100101
01110010
01110100
01100001
01101001
01101110
00100000
01100011
01101111
01101101
01110000
01101111
01110101
01101110
01100100
01110011
00100000
01101001
01101110
00100000
01110100
01101000
01100101
00100000
01100010
01110010
01100101
01110111
00100000
01100001
01110010
01100101
00100000
01100010
01100101
01101110
01100101
01100110
01101001
01100011
01101001
01100001
01101100
00101110
00001010
00001010
01011001
01101111
01110101
00100000
01101101
01100001
01111001
00100000
01110100
01101000
01101001
01101110
01101011
00100000
01101111
01100110
00100000
01100011
01101111
01100110
01100110
01100101
01100101
00100000
01100001
01110011
0

'Can coffee help you live longer?\n\nCertain compounds in the brew are beneficial.\n\nYou may think of coffee as just a part of your morning routine. But it may be part of a longer, healthier life. A study published in The New England Journal of Medicine found that among older adults, those who drank coffee (caffeinated or decaf) had a lower risk of dying from diabetes, heart disease, respiratory disease, and other medical complications than non-coffee drinkers. "I think the evidence is pretty substantial now; it seems to be beneficial across the board," says Dr. Eric Rimm, associate professor of epidemiology and nutrition at the Harvard School of Public Health.\n\nWhat they found:\n\nResearchers looked at survey responses regarding the coffee habits of more than 400,000 older men and women. After adjusting for the effects of other risk factors such as alcohol consumption and smoking, scientists concluded that two or more cups of coffee per day equated to a 10% reduction in overall dea

In [20]:
#
#

filename = 'flag_with_coffee_article.png'   # feel free to change this
image_rgb = getRGB(filename)
message = desteganographize(image_rgb)
print(f"message is {message}")

Reading image from 'flag_with_coffee_article.png':
  Format: PNG
  Original Size: (625, 415)
  Mode: RGB
There are 415 rows and 625 columns

01000011
01100001
01101110
00100000
01100011
01101111
01100110
01100110
01100101
01100101
00100000
01101000
01100101
01101100
01110000
00100000
01111001
01101111
01110101
00100000
01101100
01101001
01110110
01100101
00100000
01101100
01101111
01101110
01100111
01100101
01110010
00111111
00001010
00001010
01000011
01100101
01110010
01110100
01100001
01101001
01101110
00100000
01100011
01101111
01101101
01110000
01101111
01110101
01101110
01100100
01110011
00100000
01101001
01101110
00100000
01110100
01101000
01100101
00100000
01100010
01110010
01100101
01110111
00100000
01100001
01110010
01100101
00100000
01100010
01100101
01101110
01100101
01100110
01101001
01100011
01101001
01100001
01101100
00101110
00001010
00001010
01011001
01101111
01110101
00100000
01101101
01100001
01111001
00100000
01110100
01101000
01101001
01101110
01101011
00100000
0110

In [39]:
def steganographize( image_rgb, message ):
    """Embeds a message in an image's RGB data"""
    
    num_rows = len(image_rgb) 
    num_cols = len(image_rgb[0])
    print(f"There are {num_rows} rows and {num_cols} columns\n")
    
    #
    # some helper code to convert to individual zeros and ones...
    #
    binaryMessage = ""
    for char in message:
        value = ord(char)
        binary = bin(value)[2:]
        nbits = len(binary)    # to make sure we have 8 bits...
        binary = '0'*(8-nbits) + binary
        binaryMessage += binary
    binaryMessage += '00000000'
    
    # example message: 01010110110
    
    ml = len(binaryMessage)
    available = num_rows*num_cols*3
    print(f"Message in binary is {binaryMessage} (with {ml//3} full pixels and {ml%3} extra values)")
    print(f"This image has {available} pixels\n")
    
    if ml > available:
        print(f"There is not enough space to encode your message here...")
        return 42
    
    new_rgb = copy.deepcopy(image_rgb)
    BM = 0
    pointer = 0
    
    for row in range(num_rows):
        # print(f"row is {row}")
        for col in range(num_cols):
            for index, bit in enumerate(image_rgb[row][col]): # this bit is the rgb number
                
                # check if pointer is less than the legnth of the binary message
                if pointer < len(binaryMessage):
                    
                    # checking if we need to change the rgb value
                    if bit%2 != int(binaryMessage[pointer]): 
                        bit = bit+1
                    
                    # since tuples are immutable, we are changing the whole tuple itself
                    if index == 0:
                        new_rgb[row][col] = (bit, new_rgb[row][col][1],  new_rgb[row][col][2])
                    elif index == 1:
                        new_rgb[row][col] = (new_rgb[row][col][0], bit, new_rgb[row][col][2])
                    else:
                        new_rgb[row][col] = (new_rgb[row][col][0], new_rgb[row][col][1], bit)
                    
                    pointer += 1
                    
                else:
                   return new_rgb 
    return new_rgb

In [48]:
filename = "flag.png"   # the image to embed the message in
message = "hello world"   # the message

image_rgb = getRGB(filename)
image_new = steganographize(image_rgb, message)

Reading image from 'flag.png':
  Format: PNG
  Original Size: (625, 415)
  Mode: RGB
There are 415 rows and 625 columns

Message in binary is 011010000110010101101100011011000110111100100000011101110110111101110010011011000110010000000000 (with 32 full pixels and 0 extra values)
This image has 778125 pixels

(48, 130, 160)
(48, 131, 160)
(48, 131, 161)
(48, 132, 163)
(48, 133, 163)
(48, 133, 164)
(46, 134, 166)
(46, 134, 166)
(46, 134, 166)
(45, 135, 166)
(45, 135, 166)
(45, 135, 166)
(44, 135, 166)
(44, 135, 166)
(44, 135, 166)
(43, 135, 168)
(43, 136, 168)
(43, 136, 169)
(41, 134, 168)
(41, 134, 168)
(41, 134, 169)
(41, 134, 168)
(41, 134, 168)
(41, 134, 168)
(38, 135, 168)
(38, 135, 168)
(38, 135, 169)
(36, 133, 166)
(36, 133, 166)
(36, 133, 167)
(36, 135, 169)
(36, 136, 169)
(36, 136, 170)
(33, 136, 171)
(33, 137, 171)
(33, 137, 172)
(33, 137, 174)
(33, 137, 174)
(33, 137, 175)
(33, 136, 173)
(33, 136, 173)
(33, 136, 174)
(33, 136, 173)
(33, 136, 173)
(33, 136, 174)
(34, 137, 176)


In [46]:
# choose a name for the image you will create
savefile = "image_with_message.png"   # save the _new_ image

saveRGB(image_new, savefile)

Starting to save image_with_message.png ...
image_with_message.png saved.


In [47]:
image_rgb = getRGB(savefile)
message = desteganographize(image_rgb)
print(f"message is {message}")

Reading image from 'image_with_message.png':
  Format: PNG
  Original Size: (625, 415)
  Mode: RGB
There are 415 rows and 625 columns

01101000
01100101
01101100
01101100
01101111
00100000
01110111
01101111
01110010
01101100
01100100
message is hello world


In [67]:
filename = "theoffice.png"   # the image to embed the message in
message = "I talk a lot so Ive learned to tune myself out. Kelly Kapoor"  # the message

image_rgb = getRGB(filename)
image_new = steganographize(image_rgb, message)

Reading image from 'theoffice.png':
  Format: WEBP
  Original Size: (500, 667)
  Mode: RGB
There are 667 rows and 500 columns

Message in binary is 01001001001000000111010001100001011011000110101100100000011000010010000001101100011011110111010000100000011100110110111100100000010010010111011001100101001000000110110001100101011000010111001001101110011001010110010000100000011101000110111100100000011101000111010101101110011001010010000001101101011110010111001101100101011011000110011000100000011011110111010101110100001011100010000001001011011001010110110001101100011110010010000001001011011000010111000001101111011011110111001000000000 (with 162 full pixels and 2 extra values)
This image has 1000500 pixels

(194, 193, 197)
(194, 193, 197)
(194, 193, 198)
(192, 191, 195)
(192, 191, 195)
(192, 191, 196)
(190, 189, 193)
(190, 189, 193)
(190, 189, 194)
(190, 187, 192)
(190, 187, 192)
(190, 187, 192)
(188, 184, 189)
(188, 184, 189)
(188, 184, 190)
(182, 180, 184)
(182, 180, 184)
(182, 180, 185)
(1

In [68]:
# choose a name for the image you will create
savefile = "office_with_message1.png"   # save the _new_ image

saveRGB(image_new, savefile)

Starting to save office_with_message1.png ...
office_with_message1.png saved.


In [69]:
image_rgb = getRGB(savefile)
message = desteganographize(image_rgb)
print(f"message is {message}")

Reading image from 'office_with_message1.png':
  Format: PNG
  Original Size: (500, 667)
  Mode: RGB
There are 667 rows and 500 columns

01001001
00100000
01110100
01100001
01101100
01101011
00100000
01100001
00100000
01101100
01101111
01110100
00100000
01110011
01101111
00100000
01001001
01110110
01100101
00100000
01101100
01100101
01100001
01110010
01101110
01100101
01100100
00100000
01110100
01101111
00100000
01110100
01110101
01101110
01100101
00100000
01101101
01111001
01110011
01100101
01101100
01100110
00100000
01101111
01110101
01110100
00101110
00100000
01001011
01100101
01101100
01101100
01111001
00100000
01001011
01100001
01110000
01101111
01101111
01110010
message is I talk a lot so Ive learned to tune myself out. Kelly Kapoor


## Extra Credit - Image Inception!

For this extra credit assignment, write two functions that embed and extract a message from an image ... only this time the message will be an IMAGE

Note that the image you embed must be (significantly) smaller than the image you are embedding it in!

I (aka Kanalu!) would personally recommend embedding a greyscale image ... The reason is that instead of having a list-of-list-of-lists (each pixel has an r, g, and b value), a greyscale image only needs _one_ value per pixel. And yes, the `show_image()` function can read a simple list-of-lists for greyscale images! Much less data will be required for such an image

We have provided two functions to convert RGB data to greyscale data...
And, a bit of additional code.

HOWEVER, as always, it is completely up to you how you want to approach this problem! Feel free to embed small color images anyway!

In [ ]:
def greyscaleHelp( rgbpixel ):
    """ Helper Function
        rgbpixel should be in the form [r,g,b]
        returns [newred, newgreen, new blue],
        based on their old versions!
    """
    [r,g,b] = rgbpixel
    lum = (21 * r)//100 + (72 * g)//100 + (7 * b)//100   # a generic formula to convert rgb to greyscale
    return lum   # returns a single number

In [ ]:
def greyscale( image ):
    """Makes an image grayscale"""
    
    new_image = [[ greyscaleHelp(pix) for pix in row] for row in image]   # sick list comprehension
    return new_image

In [ ]:
filename = "flag.png"   # image to greyscale-ify

rgb = getRGB(filename)
grey_filename = greyscale(rgb)
print(f"Here's part of the first row of pixels: {grey_filename[0][0:10]}...")
show_image(grey_filename)

In [ ]:
resizeMe = "flag.jpg"   # image to resize
newImage = "smallflag.jpg"   # name for resized image (new)

#
# rescaling
#
original = Image.open(resizeMe)
print(f"Reading image from '{resizeMe}':")
print(f"  Format: {original.format}\n  Original Size: {original.size}\n  Mode: {original.mode}")
max_dim = max(original.size)
scale = max_dim/200   # maximum pixel dimension is 200
new_size = tuple([round(x/scale) for x in original.size])
new = original.resize(new_size,Image.ANTIALIAS)
new.save(newImage,optimize=True, quality=95)
print(f"New size is {new_size}")
show_image(getRGB(newImage))

In [ ]:
def steg_image(original, embed):
    """Embeds an image (embded) in another image's (original) RGB data"""
    
    num_rows = len(original) 
    num_cols = len(original[0])
    
    R = len(embed)
    C = len(embed[0])
    
    print(f"\nThere are {num_rows} rows and {num_cols} columns in the original image")
    print(f"There are {R} rows and {C} columns in the embedded image")
    
    available = num_rows*num_cols*3
    
    print(f"Embedded greyscale image has {R*C*8} bits")
    print(f"This image has {available} pixel values\n")
    
    if R*C*8 > available:
        print(f"There is not enough space to encode your message here...")
        return 42
    
    # work to be done here!

In [ ]:
def desteg_image(steg):
    """Extracts an image hidden in an image's RGB data"""
    
    num_rows = len(steg) 
    num_cols = len(steg[0])
    print(f"There are {num_rows} rows and {num_cols} columns\n")
    sub = ""
    newImage = []
    end = False
    
    for row in range(num_rows):
        for col in range(num_cols):
            pass
            # work to be done here!

In [ ]:
# file1 = "flag.jpg"   # main image
# file2 = "smallflag.jpg"   # embedded image

# original = getRGB(file1)
# embed = getRGB(file2)

# steg = steg_image(original, embed)

# show_image(steg)

# print("\n[DESTEGGING]\n")
# desteg = desteg_image(steg)
# print(f"There is {len(desteg)} desteg data")
# print(f"Here is some of it: {desteg[:42]} ... {desteg[-42:]}")

# show_image(desteg.reshape(Image.open(file2).size))

In [ ]:
# savefile = "funny.png"   # save the _new_ image
# saveRGB(steg, savefile)
# show_image(desteg_image(getRGB(savefile)).reshape(Image.open(file2).size))   # and open the embedded image